# 実世界ユースケースのためのモデル選択実践ガイド

## 目的と対象読者

このクックブックは、特定のワークロードに対して適切なOpenAIモデル（GPT 4.1、o3、o4-mini）を選択、プロンプト、デプロイするための実践的なガイドです。網羅的なドキュメントではなく、ソリューションエンジニア、テクニカルアカウントマネージャー、パートナーアーキテクト、および半技術的な実務者が迅速に動作するソリューションを構築するのに役立つ実用的な意思決定フレームワークと実世界の例を提供します。コンテンツは現在のモデル機能、業界特化の実装、今日の業界ニーズに焦点を当て、モデル選択から本番デプロイまでの明確な道筋を示します。各セクションでは、特定のトピックについてより深く掘り下げるための既存リソースを参照しながら、あなたのユースケースにすぐに適用できる簡潔で適応可能なコード例を提供します。

> 注：以下の規範的ガイダンスと実験は、現在利用可能な最新のSOTAモデルで実施されています。これらの指標は、異なるシナリオとタイムラインを考慮して、将来変更される可能性があります。

## このクックブックの使い方

このクックブックは、必要な情報を素早く見つけられるよう、明確なセクションに分かれています。各セクションは、モデル選択、実装、デプロイの特定の側面をカバーしています。

1. **[目的と対象読者](#purpose-audience)**：このクックブックの対象者と内容の概要。
2. **[モデルガイド](#model-guide)**：ニーズに適したモデルを選択するためのクイックリファレンス。異なるユースケースシナリオのマッピングに基づくモデル比較と進化図を含む。
3. **ユースケース**：
   - **[3A. 法的Q&Aのための長コンテキストRAG](#3a-use-case-long-context-rag-for-legal-qa)**：複雑な法的文書から質問に答えるエージェントシステムの構築。
   - **[3B. 製薬R&DのためのAI共同研究者](#3b-use-case-ai-co-scientist-for-pharma-rd)**：マルチエージェントシステムによる製薬研究の実験設計の加速。
   - **[3C. 保険請求処理](#3c-use-case-insurance-claim-processing)**：手書きの保険フォームのデジタル化と検証（ビジョンと推論を使用）。
4. **[プロトタイプから本番へ](#prototype-to-production)**：プロトタイプから本番への移行を支援するチェックリスト。
5. **[適応決定ツリー](#adaptation-decision-tree)**：特定の要件に基づくモデル選択を導くフローチャート。
6. **[付録](#appendices)**：価格、レイテンシ、プロンプトパターン、外部リソースへのリンクを含む参考資料。

迅速な決定には、モデルガイドと適応決定ツリーのセクションに焦点を当ててください。実装の詳細については、あなたのニーズに関連する特定のユースケースを探索してください。

================================================================================

## モデルガイド

## 2.1 モデル紹介マトリックス

| モデル | 中核的強み | 理想的な最初の選択肢 | 注意点 | エスカレーション/ダウングレードパス |
| :---- | :---- | :---- | :---- | :---- |
| GPT‑4o | リアルタイム音声/ビジョンチャット | ライブマルチモーダルエージェント | テキストSOTA（最先端）では4.1よりわずかに劣る | 深い推論が必要 → o4‑mini |
| GPT‑4.1 | 100万トークンテキスト精度の王者 | 長文書解析、コードレビュー | ネイティブに推論できない；miniより高コスト | 予算制約 → 4.1‑mini / nano |
| o3 | 深いツール使用エージェント | 高リスク、マルチステップ推論 | レイテンシと価格 | コスト/レイテンシ → o4‑mini |
| o4‑mini | 安価で高速な推論 | 大量の「十分に良い」ロジック | o3と比較した深度の上限 | 精度が重要 → o3 |

*（完全な価格とユーティリティテーブル → [セクション6.1](#appendices)）*

## 2.2 モデル進化の概観

OpenAIのモデルラインナップは、異なる次元での専門的なニーズに対応するよう進化してきました。これらの図は、現在のモデルファミリーとその関係を示しています。

### 基本的な違い：「oシリーズ」vs「GPT」モデル

OpenAIは、それぞれ独自の強みを持つ2つの異なるモデルファミリーを提供しています：

- **GPTモデル（4o、4.1）**：優れた指示追従能力を持つ汎用タスクに最適化。GPT-4.1は長いコンテキスト（100万トークン）で優秀で、GPT-4oはリアルタイム音声、テキスト読み上げ、音声認識のバリアントがあります。GPT-4.1にはminiとnanoバリアントもあり、GPT-4oにはminiバリアントがあります。これらのバリアントは、フルサイズの対応モデルより安価で高速です。

- **oシリーズモデル（o3、o4-mini）**：深い推論とステップバイステップの問題解決に特化。これらのモデルは、論理的思考とツール使用を必要とする複雑なマルチステージタスクで優秀です。精度と推論の深さが最重要の場合に選択してください。これらのモデルには、オプションの`reasoning_effort`パラメータ（`low`、`medium`、`high`に設定可能）があり、ユーザーが推論に使用するトークン量を制御できます。

### OpenAIモデル進化

![OpenAIモデル進化](../../../images/2.2_model_evolution.png)

### 主要特性

- **GPT-4.1ファミリー**：100万トークンのコンテキストウィンドウで長いコンテキスト処理に最適化。
- **o3**：深いマルチステップ推論に特化。
- **o4-mini**：推論能力とビジョンを低コストで組み合わせ。

各モデルは異なるシナリオで優秀で、複雑なワークフローに組み合わせることができる補完的な強みを持っています。

このクックブックでは、GPT-4.1シリーズモデル、o3、o4-miniのみで実験しました。GPT-4oシリーズモデルでは実験していません。

================================================================================

## 3A. ユースケース：法的Q&Aのための長コンテキストRAG

![法的Q&Aのための長コンテキストRAG](../../../images/3A_rag_task_card.png)
## 🗂️ TL;DRマトリックス

この表は、**この特定の長コンテキストエージェントRAG実装**の中核技術選択とその根拠をまとめています。

| レイヤー | 選択 | ユーティリティ |
| :---- | :---- | :---- |
| **チャンキング** | 文境界認識スプリッター | 文境界を尊重して文書を20の等しいチャンクに分割。 |
| **ルーティング** | `gpt-4.1-mini` | 埋め込みインデックスなしで自然言語理解を使用して関連チャンクを特定。 |
| **パス選択** | `select(ids=[...])` と `scratchpad(text="...")` | 文書階層を掘り下げながら推論を記録。 |
| **引用** | 段落レベル | 精度とコストのバランス；回答に意味のあるコンテキストを提供。 |
| **統合** | `gpt-4.1`（構造化出力） | 選択された段落から引用付きで直接回答を生成。 |
| **検証** | `o4-mini`（LLM-as-Judge） | 事実の正確性と引用の正確性を検証。 |

*注：価格とモデル識別子は2025年4月時点で正確、変更される可能性があります。*

このセクションでは、*商標審判部手続きマニュアル（TBMP）*を代表的なケースとして使用し、複雑で長い手続き文書について正確に質問に答えるよう設計されたRetrieval-Augmented Generation（RAG）システムの構築について説明します。TBMPは、USPTO商標審判部での商標訴訟を管理する手続きを詳述する重要な法的リソースであり、知的財産弁護士や法的専門家によって頻繁に参照されます。最新のOpenAIモデルを活用することで、システムは密度の高い法的コンテンツの理解と解釈可能性を向上させ、高度な言語理解と動的検索機能を通じて正確でコンテキストを認識した応答を可能にします。

これらのアプローチは、医療コンプライアンスマニュアル、金融規制フレームワーク、または精度、引用、監査可能性がミッションクリティカルな要件である技術文書システムなど、複雑な文書からの正確な情報検索を必要とする他のユースケースにも適用できます。

## 1. シナリオスナップショット

* **コーパス：** 主要文書は[商標審判部手続きマニュアル（TBMP、2024年版）](https://www.uspto.gov/sites/default/files/documents/tbmp-Master-June2024.pdf)です。このマニュアルには詳細な手続き規則とガイドラインが含まれ、総ページ数は1194ページです。
* **ユーザー：** 対象ユーザーは、TBMPに*のみ*基づく手続き質問に対する迅速で正確な回答を必要とする知的財産（IP）訴訟アソシエイトとパラリーガルです。
* **典型的な質問：** ユーザーは統合と引用を必要とする質問を投げかけます：
  1. "TBMPによる証拠開示を強制する申立ての提出要件は何ですか？"
  2. "マニュアルで指定されている証拠開示会議に適用される期限は何ですか？"
  3. "TBMPによると、証言録取中の弁護士・依頼者間秘匿特権の主張を審判部はどのように処理しますか？"
  4. "TBMPによると、審判部が発動できるFed. R. Civ. P. 11制裁を列挙してください。"

*注：特定のデプロイメント環境によっては、インフラ要件に合わせて一部の実装ステップを適応させる必要がある場合があります。*

> OpenAIのFile Searchツールは多くのユースケースに良い出発点を提供しますが、このセクションでは、前処理やベクトルデータベースなしで大きな文書を処理するために100万トークンのコンテキストウィンドウを活用する異なるアプローチを紹介します。ここで説明するエージェントアプローチは、ゼロレイテンシ取り込み、動的な検索粒度、細かい引用追跡可能性を可能にします。

## 2. エージェントRAGフロー

実装に入る前に、全体的なアプローチを理解しましょう：

1. **文書全体をロード**してコンテキストウィンドウに入れる
2. **文境界を尊重して20チャンクに分割**
3. **モデルに質問**してどのチャンクが関連情報を含む可能性があるかを特定
4. **選択されたチャンクを掘り下げ**てさらに分割
5. **段落レベルのコンテンツに到達するまで繰り返し**
6. **選択された段落に基づいて回答を生成**
7. **事実の正確性について回答を検証**

この階層ナビゲーションアプローチは、人間が文書をざっと読み、関連する章に焦点を当て、次に特定のセクション、最後に最も関連する段落のみを読む方法を模倣しています。

![Hierarchical Router](../../../images/3A_rag_hierarchical_router.png)


## エージェント型RAGシステム：モデル使用方法

| プロセス段階 | 使用モデル | 目的 |
|---------------|------------|---------|
| 初期ルーティング | `gpt-4.1-mini` | 関連情報を含む可能性のある文書チャンクを特定 |
| 階層ナビゲーション | `gpt-4.1-mini` | 最も関連性の高い段落を見つけるために掘り下げを継続 |
| 回答生成 | `gpt-4.1` | 選択された段落からの引用を含む構造化された回答を作成 |
| 回答検証 | `o4-mini` | 事実の正確性と適切な引用使用を検証 |

この前処理不要のアプローチは、大きなコンテキストウィンドウを活用して文書をその場でナビゲートし、人間が文書をざっと読んで関連情報を見つける方法を模倣します。

## 3. 実装

このアプローチを段階的に実装していきましょう。

まず、必要なパッケージをインストールします。

In [1]:
%pip install tiktoken pypdf nltk openai pydantic --quiet

Note: you may need to restart the kernel to use updated packages.


### 3.1 文書の読み込み

まず、文書を読み込んでそのサイズを確認しましょう。このガイドでは、セクション100-900に焦点を当てます。これらのセクションは、委員会決定の審査を通じて、中核的な手続き的側面をカバーしています。セクション1000以降（干渉、併用手続き、一方的審判）は、現在の範囲外の専門的な手続きです。

In [3]:
import requests
from io import BytesIO
from pypdf import PdfReader
import re
import tiktoken
from nltk.tokenize import sent_tokenize
import nltk
from typing import List, Dict, Any

# Download nltk data if not already present
nltk.download('punkt_tab')

def load_document(url: str) -> str:
    """Load a document from a URL and return its text content."""
    print(f"Downloading document from {url}...")
    response = requests.get(url)
    response.raise_for_status()
    pdf_bytes = BytesIO(response.content)
    pdf_reader = PdfReader(pdf_bytes)
    
    full_text = ""
    

    max_page = 920  # Page cutoff before section 1000 (Interferences)
    for i, page in enumerate(pdf_reader.pages):
        if i >= max_page:
            break
        full_text += page.extract_text() + "\n"
    
    # Count words and tokens
    word_count = len(re.findall(r'\b\w+\b', full_text))
    
    tokenizer = tiktoken.get_encoding("o200k_base")
    token_count = len(tokenizer.encode(full_text))
    
    print(f"Document loaded: {len(pdf_reader.pages)} pages, {word_count} words, {token_count} tokens")
    return full_text

# Load the document
tbmp_url = "https://www.uspto.gov/sites/default/files/documents/tbmp-Master-June2024.pdf"
document_text = load_document(tbmp_url)

# Show the first 500 characters
print("\nDocument preview (first 500 chars):")
print("-" * 50)
print(document_text[:500])
print("-" * 50)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/kmurali/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Document loaded: 1194 pages, 595197 words, 932964 tokens

Document preview (first 500 chars):
--------------------------------------------------
TRADEMARK TRIAL AND
APPEAL BOARD MANUAL
OF PROCEDURE (TBMP)
 June 2024
June   2024
United States Patent and Trademark Office
PREFACE TO THE JUNE 2024 REVISION
The June 2024 revision of the Trademark Trial and Appeal Board Manual of Procedure is an update of the
June 2023 edition. This update is moderate in nature and incorporates relevant case law issued between March
3, 2023 and March 1, 2024.
The title of the manual is abbreviated as “TBMP.” A citation to a section of the manual may be written
--------------------------------------------------


このドキュメントは90万トークンを超える長さであることがわかります！GPT 4.1のコンテキスト長に収めることは可能ですが、検証可能な引用も必要なので、再帰的チャンク戦略で進めていきます。

### 3.2 最小トークンサイズを持つ改良された20チャンク分割器

次に、文書を20個のチャンクに分割する改良された関数を作成しましょう。各チャンクが最小トークンサイズを持ち、文の境界を尊重することを保証します。

> 20という数字は、この特定の文書/タスクに対して経験的に選択された数値であり、サイズや構造に基づいて他の文書では調整が必要かもしれません（数値が高いほど、より細かいチャンクになります）。しかし、ここでの重要な原則は、言語モデルが関連するコンポーネントを決定できるように、文書のセクションを分割することです。この同じ理由は、このクックブックで後に紹介される`max_depth`パラメータにも適用されます。

In [4]:
# Global tokenizer name to use consistently throughout the code
TOKENIZER_NAME = "o200k_base"

def split_into_20_chunks(text: str, min_tokens: int = 500) -> List[Dict[str, Any]]:
    """
    Split text into up to 20 chunks, respecting sentence boundaries and ensuring
    each chunk has at least min_tokens (unless it's the last chunk).
    
    Args:
        text: The text to split
        min_tokens: The minimum number of tokens per chunk (default: 500)
    
    Returns:
        A list of dictionaries where each dictionary has:
        - id: The chunk ID (0-19)
        - text: The chunk text content
    """
    # First, split the text into sentences
    sentences = sent_tokenize(text)
    
    # Get tokenizer for counting tokens
    tokenizer = tiktoken.get_encoding(TOKENIZER_NAME)
    
    # Create chunks that respect sentence boundaries and minimum token count
    chunks = []
    current_chunk_sentences = []
    current_chunk_tokens = 0
    
    for sentence in sentences:
        # Count tokens in this sentence
        sentence_tokens = len(tokenizer.encode(sentence))
        
        # If adding this sentence would make the chunk too large AND we already have the minimum tokens,
        # finalize the current chunk and start a new one
        if (current_chunk_tokens + sentence_tokens > min_tokens * 2) and current_chunk_tokens >= min_tokens:
            chunk_text = " ".join(current_chunk_sentences)
            chunks.append({
                "id": len(chunks),  # Integer ID instead of string
                "text": chunk_text
            })
            current_chunk_sentences = [sentence]
            current_chunk_tokens = sentence_tokens
        else:
            # Add this sentence to the current chunk
            current_chunk_sentences.append(sentence)
            current_chunk_tokens += sentence_tokens
    
    # Add the last chunk if there's anything left
    if current_chunk_sentences:
        chunk_text = " ".join(current_chunk_sentences)
        chunks.append({
            "id": len(chunks),  # Integer ID instead of string
            "text": chunk_text
        })
    
    # If we have more than 20 chunks, consolidate them
    if len(chunks) > 20:
        # Recombine all text
        all_text = " ".join(chunk["text"] for chunk in chunks)
        # Re-split into exactly 20 chunks, without minimum token requirement
        sentences = sent_tokenize(all_text)
        sentences_per_chunk = len(sentences) // 20 + (1 if len(sentences) % 20 > 0 else 0)
        
        chunks = []
        for i in range(0, len(sentences), sentences_per_chunk):
            # Get the sentences for this chunk
            chunk_sentences = sentences[i:i+sentences_per_chunk]
            # Join the sentences into a single text
            chunk_text = " ".join(chunk_sentences)
            # Create a chunk object with ID and text
            chunks.append({
                "id": len(chunks),  # Integer ID instead of string
                "text": chunk_text
            })
    
    # Print chunk statistics
    print(f"Split document into {len(chunks)} chunks")
    for i, chunk in enumerate(chunks):
        token_count = len(tokenizer.encode(chunk["text"]))
        print(f"Chunk {i}: {token_count} tokens")
    
    return chunks

# Split the document into 20 chunks with minimum token size
document_chunks = split_into_20_chunks(document_text, min_tokens=500)

Split document into 20 chunks
Chunk 0: 42326 tokens
Chunk 1: 42093 tokens
Chunk 2: 42107 tokens
Chunk 3: 39797 tokens
Chunk 4: 58959 tokens
Chunk 5: 48805 tokens
Chunk 6: 37243 tokens
Chunk 7: 33453 tokens
Chunk 8: 38644 tokens
Chunk 9: 49402 tokens
Chunk 10: 51568 tokens
Chunk 11: 49586 tokens
Chunk 12: 47722 tokens
Chunk 13: 48952 tokens
Chunk 14: 44994 tokens
Chunk 15: 50286 tokens
Chunk 16: 54424 tokens
Chunk 17: 62651 tokens
Chunk 18: 47430 tokens
Chunk 19: 42507 tokens


### 3.3 改良されたツールスキーマを使用したルーター関数

次に、関連するチャンクを選択し、スクラッチパッドを維持するルーター関数を作成しましょう。

> スクラッチパッドを維持することで、モデルが時間の経過とともに決定基準と推論を追跡できるようになります。この実装では、GPT-4.1-miniを使用した2パスアプローチを採用しています：最初にツール呼び出し（tool_choice="required"）を通じてモデルにスクラッチパッドの更新を要求し、次にチャンク選択のための構造化されたJSON出力を要求します。このアプローチにより、モデルの推論プロセスの可視性を向上させながら、下流処理のための一貫した構造化出力を確保します。

In [5]:
from openai import OpenAI
import json
from typing import List, Dict, Any

# Initialize OpenAI client
client = OpenAI()

def route_chunks(question: str, chunks: List[Dict[str, Any]], 
                depth: int, scratchpad: str = "") -> Dict[str, Any]:
    """
    Ask the model which chunks contain information relevant to the question.
    Maintains a scratchpad for the model's reasoning.
    Uses structured output for chunk selection and required tool calls for scratchpad.
    
    Args:
        question: The user's question
        chunks: List of chunks to evaluate
        depth: Current depth in the navigation hierarchy
        scratchpad: Current scratchpad content
    
    Returns:
        Dictionary with selected IDs and updated scratchpad
    """
    print(f"\n==== ROUTING AT DEPTH {depth} ====")
    print(f"Evaluating {len(chunks)} chunks for relevance")
    
    # Build system message
    system_message = """You are an expert document navigator. Your task is to:
1. Identify which text chunks might contain information to answer the user's question
2. Record your reasoning in a scratchpad for later reference
3. Choose chunks that are most likely relevant. Be selective, but thorough. Choose as many chunks as you need to answer the question, but avoid selecting too many.

First think carefully about what information would help answer the question, then evaluate each chunk.
"""

    # Build user message with chunks and current scratchpad
    user_message = f"QUESTION: {question}\n\n"
    
    if scratchpad:
        user_message += f"CURRENT SCRATCHPAD:\n{scratchpad}\n\n"
    
    user_message += "TEXT CHUNKS:\n\n"
    
    # Add each chunk to the message
    for chunk in chunks:
        user_message += f"CHUNK {chunk['id']}:\n{chunk['text']}\n\n"
    
    # Define function schema for scratchpad tool calling
    tools = [
        {
            "type": "function",
            "name": "update_scratchpad",
            "description": "Record your reasoning about why certain chunks were selected",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "Your reasoning about the chunk(s) selection"
                    }
                },
                "required": ["text"],
                "additionalProperties": False
            }
        }
    ]
    
    # Define JSON schema for structured output (selected chunks)
    text_format = {
        "format": {
            "type": "json_schema",
            "name": "selected_chunks",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                    "chunk_ids": {
                        "type": "array",
                        "items": {"type": "integer"},
                        "description": "IDs of the selected chunks that contain information to answer the question"
                    }
                },
                "required": [
                    "chunk_ids"
                ],
                "additionalProperties": False
            }
        }
    }
    
    # First pass: Call the model to update scratchpad (required tool call)
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message + "\n\nFirst, you must use the update_scratchpad function to record your reasoning."}
    ]
    
    response = client.responses.create(
        model="gpt-4.1-mini",
        input=messages,
        tools=tools,
        tool_choice="required"
    )
    
    # Process the scratchpad tool call
    new_scratchpad = scratchpad
    
    for tool_call in response.output:
        if tool_call.type == "function_call" and tool_call.name == "update_scratchpad":
            args = json.loads(tool_call.arguments)
            scratchpad_entry = f"DEPTH {depth} REASONING:\n{args.get('text', '')}"
            if new_scratchpad:
                new_scratchpad += "\n\n" + scratchpad_entry
            else:
                new_scratchpad = scratchpad_entry
            
            # Add function call and result to messages
            messages.append(tool_call)
            messages.append({
                "type": "function_call_output",
                "call_id": tool_call.call_id,
                "output": "Scratchpad updated successfully."
            })
    
    # Second pass: Get structured output for chunk selection
    messages.append({"role": "user", "content": "Now, select the chunks that could contain information to answer the question. Return a JSON object with the list of chunk IDs."})
    
    response_chunks = client.responses.create(
        model="gpt-4.1-mini",
        input=messages,
        text=text_format
    )
    
    # Extract selected chunk IDs from structured output
    selected_ids = []
    if response_chunks.output_text:
        try:
            # The output_text should already be in JSON format due to the schema
            chunk_data = json.loads(response_chunks.output_text)
            selected_ids = chunk_data.get("chunk_ids", [])
        except json.JSONDecodeError:
            print("Warning: Could not parse structured output as JSON")
    
    # Display results
    print(f"Selected chunks: {', '.join(str(id) for id in selected_ids)}")
    print(f"Updated scratchpad:\n{new_scratchpad}")
    
    return {
        "selected_ids": selected_ids,
        "scratchpad": new_scratchpad
    }

### 3.4 再帰的ナビゲーション関数

次に、ドキュメントを掘り下げていく再帰的ナビゲーション関数を作成しましょう。`max_depth`は掘り下げる最大レベル数です（トークンの最小値を念頭に置いて）：

In [6]:
def navigate_to_paragraphs(document_text: str, question: str, max_depth: int = 1) -> Dict[str, Any]:
    """
    Navigate through the document hierarchy to find relevant paragraphs.
    
    Args:
        document_text: The full document text
        question: The user's question
        max_depth: Maximum depth to navigate before returning paragraphs (default: 1)
    
    Returns:
        Dictionary with selected paragraphs and final scratchpad
    """
    scratchpad = ""
    
    # Get initial chunks with min 500 tokens
    chunks = split_into_20_chunks(document_text, min_tokens=500)
    
    # Navigator state - track chunk paths to maintain hierarchy
    chunk_paths = {}  # Maps numeric IDs to path strings for display
    for chunk in chunks:
        chunk_paths[chunk["id"]] = str(chunk["id"])
    
    # Navigate through levels until max_depth or until no chunks remain
    for current_depth in range(max_depth + 1):
        # Call router to get relevant chunks
        result = route_chunks(question, chunks, current_depth, scratchpad)
        
        # Update scratchpad
        scratchpad = result["scratchpad"]
        
        # Get selected chunks
        selected_ids = result["selected_ids"]
        selected_chunks = [c for c in chunks if c["id"] in selected_ids]
        
        # If no chunks were selected, return empty result
        if not selected_chunks:
            print("\nNo relevant chunks found.")
            return {"paragraphs": [], "scratchpad": scratchpad}
        
        # If we've reached max_depth, return the selected chunks
        if current_depth == max_depth:
            print(f"\nReturning {len(selected_chunks)} relevant chunks at depth {current_depth}")
            
            # Update display IDs to show hierarchy
            for chunk in selected_chunks:
                chunk["display_id"] = chunk_paths[chunk["id"]]
                
            return {"paragraphs": selected_chunks, "scratchpad": scratchpad}
        
        # Prepare next level by splitting selected chunks further
        next_level_chunks = []
        next_chunk_id = 0  # Counter for new chunks
        
        for chunk in selected_chunks:
            # Split this chunk into smaller pieces
            sub_chunks = split_into_20_chunks(chunk["text"], min_tokens=200)
            
            # Update IDs and maintain path mapping
            for sub_chunk in sub_chunks:
                path = f"{chunk_paths[chunk['id']]}.{sub_chunk['id']}"
                sub_chunk["id"] = next_chunk_id
                chunk_paths[next_chunk_id] = path
                next_level_chunks.append(sub_chunk)
                next_chunk_id += 1
        
        # Update chunks for next iteration
        chunks = next_level_chunks

### 3.5 サンプル質問に対して改良されたナビゲーションを実行する

改良されたアプローチを使用して、サンプル質問に対してナビゲーションを実行してみましょう：

In [7]:
# Run the navigation for a sample question
question = "What format should a motion to compel discovery be filed in? How should signatures be handled?"
navigation_result = navigate_to_paragraphs(document_text, question, max_depth=2)

# Sample retrieved paragraph
print("\n==== FIRST 3 RETRIEVED PARAGRAPHS ====")
for i, paragraph in enumerate(navigation_result["paragraphs"][:3]):
    display_id = paragraph.get("display_id", str(paragraph["id"]))
    print(f"\nPARAGRAPH {i+1} (ID: {display_id}):")
    print("-" * 40)
    print(paragraph["text"])
    print("-" * 40)

Split document into 20 chunks
Chunk 0: 42326 tokens
Chunk 1: 42093 tokens
Chunk 2: 42107 tokens
Chunk 3: 39797 tokens
Chunk 4: 58959 tokens
Chunk 5: 48805 tokens
Chunk 6: 37243 tokens
Chunk 7: 33453 tokens
Chunk 8: 38644 tokens
Chunk 9: 49402 tokens
Chunk 10: 51568 tokens
Chunk 11: 49586 tokens
Chunk 12: 47722 tokens
Chunk 13: 48952 tokens
Chunk 14: 44994 tokens
Chunk 15: 50286 tokens
Chunk 16: 54424 tokens
Chunk 17: 62651 tokens
Chunk 18: 47430 tokens
Chunk 19: 42507 tokens

==== ROUTING AT DEPTH 0 ====
Evaluating 20 chunks for relevance
Selected chunks: 0, 1, 2, 3, 4, 5, 6, 7, 8
Updated scratchpad:
DEPTH 0 REASONING:
The user wants to know the format requirements for filing a motion to compel discovery and how signatures should be handled for such motions. 

Based on the evaluation of chunks:
- Chunks 0, 1, 2, 3, 4, 5, 6, 7, 8 are highly relevant since they cover general requirements for submissions, motions, signatures, service, and specifically for motions and discovery in TTAB pro

GPT 4.1-miniの結果は、スクラッチパッドがその思考プロセスを説明しながら、文書内の関連コンポーネントを反復的に抽出することを示しています！深度1では、モデルはクエリに答えるために必要な重要なコンポーネントとして「*申請書類における署名に関する詳細なルール（申立書を含む）*」と「*ESHTAの使用、'/sig/'記号方式による電子署名を含む必要な署名形式*」を特定しています。

深度2では、スクラッチパッドは電子署名に関する重要な規制を含むチャンクを正確に分離する（チャンク5-12）一方で、欠落したコンテンツを認識し続ける洗練された判断力を実証しており、「*ディスカバリー関連の申立て...は400番台以降のチャンクにあるはずです（ただし、ここでは完全に表示されていません...）*」と注記しています。

このプロセスは、GPT 4.1が法的アナリストを模倣する方法を示しており、関連コンテンツをより深く反復的に掘り下げ、その過程で推論を説明しています（これにより、モデルがなぜそのチャンクを選択したのかをデバッグしやすくなります）。

### 3.6 回答生成

次に、取得した段落を使用してGPT-4.1で回答を生成しましょう。

> ここでは、リテラルのリストを動的に構築するという巧妙なトリックを使用しています（これにより、モデルの回答を提供するオプションのいずれかに強制します -- この場合は段落ID）。提供できるオプション数にはいくつかの制限があるため、システムが500を超える文書を引用する場合、この解決策は機能しない可能性があります。その場合は、潜在的な引用を500まで制限するフィルターを設けるか、モデルに応答内で正確なIDを引用するよう求め、その後応答を後処理してIDを抽出し、引用を取得することができます（例：「... [doc 0.0.12]」と言う場合があり、正規表現を使用して引用を抽出できます）。

In [8]:
from typing import List, Dict, Any
from pydantic import BaseModel, field_validator

class LegalAnswer(BaseModel):
    """Structured response format for legal questions"""
    answer: str
    citations: List[str]
    
    @field_validator('citations')
    def validate_citations(cls, citations, info):
        # Access valid_citations from the model_config
        valid_citations = info.data.get('_valid_citations', [])
        if valid_citations:
            for citation in citations:
                if citation not in valid_citations:
                    raise ValueError(f"Invalid citation: {citation}. Must be one of: {valid_citations}")
        return citations

def generate_answer(question: str, paragraphs: List[Dict[str, Any]], 
                   scratchpad: str) -> LegalAnswer:
    """Generate an answer from the retrieved paragraphs."""
    print("\n==== GENERATING ANSWER ====")
    
    # Extract valid citation IDs
    valid_citations = [str(p.get("display_id", str(p["id"]))) for p in paragraphs]
    
    if not paragraphs:
        return LegalAnswer(
            answer="I couldn't find relevant information to answer this question in the document.",
            citations=[],
            _valid_citations=[]
        )
    
    # Prepare context for the model
    context = ""
    for paragraph in paragraphs:
        display_id = paragraph.get("display_id", str(paragraph["id"]))
        context += f"PARAGRAPH {display_id}:\n{paragraph['text']}\n\n"
    
    system_prompt = """You are a legal research assistant answering questions about the 
Trademark Trial and Appeal Board Manual of Procedure (TBMP).

Answer questions based ONLY on the provided paragraphs. Do not rely on any foundation knowledge or external information or extrapolate from the paragraphs.
Cite phrases of the paragraphs that are relevant to the answer. This will help you be more specific and accurate.
Include citations to paragraph IDs for every statement in your answer. Valid citation IDs are: {valid_citations_str}
Keep your answer clear, precise, and professional.
"""
    valid_citations_str = ", ".join(valid_citations)
    
    # Call the model using structured output
    response = client.responses.parse(
        model="gpt-4.1",
        input=[
            {"role": "system", "content": system_prompt.format(valid_citations_str=valid_citations_str)},
            {"role": "user", "content": f"QUESTION: {question}\n\nSCRATCHPAD (Navigation reasoning):\n{scratchpad}\n\nPARAGRAPHS:\n{context}"}
        ],
        text_format=LegalAnswer,
        temperature=0.3
    )
    
    # Add validation information after parsing
    response.output_parsed._valid_citations = valid_citations
    
    print(f"\nAnswer: {response.output_parsed.answer}")
    print(f"Citations: {response.output_parsed.citations}")

    return response.output_parsed

# Generate an answer
answer = generate_answer(question, navigation_result["paragraphs"], 
                       navigation_result["scratchpad"])


==== GENERATING ANSWER ====

Answer: A motion to compel discovery must be filed electronically with the Trademark Trial and Appeal Board (TTAB) through ESTTA, unless ESTTA is unavailable due to technical problems or there are extraordinary circumstances, in which case a paper submission may be permitted with a written explanation ("Documents that relate to proceedings before the Trademark Trial and Appeal Board must be filed electronically with the Board through ESTTA"; "The rules require that all submissions must be made to the Board electronically, currently through ESTTA, subject to certain limited exceptions permitting submissions to be made on paper. Any permitted paper submission must be accompanied by a written explanation showing that ESTTA was unavailable due to technical problems, or that extraordinary circumstances are present, and, where required, a Petition to the Director with the requisite petition fee" 0.0.5.0, 0.0.5.5.7.3).

The motion should include a title describin

GPT 4.1は、情報の明確な流れを維持しながら、応答全体を通して引用を効果的に統合しています。各手続き要件は特定の権威ある参考文献（「0.0.5.0」や「0.0.5.5.6.2」など）にリンクされており、情報豊富で正確にソースが示された応答を作成しています。

単に最後に引用を列挙するのではなく、各重要な要件の後に括弧内記法を使用して、引用を内容に直接織り込んでいます。このアプローチにより、規則の標準的な暗唱が、ESTTA提出手続き、電子署名要件、紙面提出の例外に関する記述が対応する規制引用によって即座に裏付けられる、十分に支持された法的分析に変換されています。

### 3.7 回答の検証

まず、引用された段落を見てみましょう：

In [9]:
cited_paragraphs = []
for paragraph in navigation_result["paragraphs"]:
    para_id = str(paragraph.get("display_id", str(paragraph["id"])))
    if para_id in answer.citations:
        cited_paragraphs.append(paragraph)
    

# Display the cited paragraphs for the audience
print("\n==== CITED PARAGRAPHS ====")
for i, paragraph in enumerate(cited_paragraphs):
    display_id = paragraph.get("display_id", str(paragraph["id"]))
    print(f"\nPARAGRAPH {i+1} (ID: {display_id}):")
    print("-" * 40)
    print(paragraph["text"])
    print("-" * 40)


==== CITED PARAGRAPHS ====

PARAGRAPH 1 (ID: 0.0.5.0):
----------------------------------------
104  Business to be Conducted in Writing
37 C.F.R. § 2.190(b)  Electronic trademark documents. … Documents that r elate to proceedings before
the Trademark Trial and Appeal Board must be filed electronically with the Board through ESTTA. 37 C.F.R. § 2.191 Action of the Office based on the written record. All business with the Office must be
transacted in writing. The action of the Office will be based exclusively on the written record. No consideration
will be given to any alleged oral promise, stipulation, or understanding when there is disagreement or doubt. With the exceptions of discovery conferences with Board participation, see TBMP § 401.01, and telephone
conferences, see TBMP § 413.01 and TBMP § 502.06, all business with the Board should be transacted in
writing. 37 C.F.R. § 2.191 . The personal attendance of parties or their attorne ys or other authorized
representatives at the off

「リテラルのリスト」トリックは、モデルに対して独自の参照を作成したりランダムなテキストをハイライトしたりするのではなく、特定の段落ID（「0.0.5.4」など）のみを引用するよう強制します。これは、GPT-4.1が選択できるデジタルな「目次」を作成するようなものと考えてください。この解決策により、正確なソース資料への検証可能な引用の軌跡を得ることができ、長いコンテキストのRAGにおける重要な問題を解決します。

最後に、LLM-as-judge アプローチで答えを検証してみましょう。

In [10]:
from typing import List, Dict, Any, Literal
from pydantic import BaseModel

class VerificationResult(BaseModel):
    """Verification result format"""
    is_accurate: bool
    explanation: str
    confidence: Literal["high", "medium", "low"]

def verify_answer(question: str, answer: LegalAnswer, 
                 cited_paragraphs: List[Dict[str, Any]]) -> VerificationResult:
    """
    Verify if the answer is grounded in the cited paragraphs.
    
    Args:
        question: The user's question
        answer: The generated answer
        cited_paragraphs: Paragraphs cited in the answer
        
    Returns:
        Verification result with accuracy assessment, explanation, and confidence level
    """
    print("\n==== VERIFYING ANSWER ====")
    
    # Prepare context with the cited paragraphs
    context = ""
    for paragraph in cited_paragraphs:
        display_id = paragraph.get("display_id", str(paragraph["id"]))
        context += f"PARAGRAPH {display_id}:\n{paragraph['text']}\n\n"
    
    # Prepare system prompt
    system_prompt = """You are a fact-checker for legal information.
Your job is to verify if the provided answer:
1. Is factually accurate according to the source paragraphs
2. Uses citations correctly

Be critical and look for any factual errors or unsupported claims.
Assign a confidence level based on how directly the paragraphs answer the question:
- high: The answer is comprehensive, accurate, and directly supported by the paragraphs
- medium: The answer is mostly accurate but may be incomplete or have minor issues
- low: The answer has significant gaps, inaccuracies, or is poorly supported by the paragraphs
"""
    
    response = client.responses.parse(
        model="o4-mini",
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"""
QUESTION: {question}

ANSWER TO VERIFY:
{answer.answer}

CITATIONS USED: {', '.join(answer.citations)}

SOURCE PARAGRAPHS:
{context}

Is this answer accurate and properly supported by the source paragraphs?
Assign a confidence level (high, medium, or low) based on completeness and accuracy.
            """}
        ],
        text_format=VerificationResult
    )
    
    # Log and return the verification result
    print(f"\nAccuracy verification: {'PASSED' if response.output_parsed.is_accurate else 'FAILED'}")
    print(f"Confidence: {response.output_parsed.confidence}")
    print(f"Explanation: {response.output_parsed.explanation}")
    
    return response.output_parsed

# Verify the answer using only the cited paragraphs
verification = verify_answer(question, answer, cited_paragraphs)

# Display final result with verification
print("\n==== FINAL VERIFIED ANSWER ====")
print(f"Verification: {'PASSED' if verification.is_accurate else 'FAILED'} | Confidence: {verification.confidence}")
print("\nAnswer:")
print(answer.answer)
print("\nCitations:")
for citation in answer.citations:
    print(f"- {citation}")


==== VERIFYING ANSWER ====

Accuracy verification: PASSED
Confidence: high
Explanation: The answer correctly states that motions to compel discovery must be filed electronically through ESTTA, with paper submissions permitted only under the limited exceptions of technical failure or extraordinary circumstances (37 C.F.R. § 2.190(b) and 2.193(b)). It accurately describes the required title and caption placement (TBMP § 105), and it appropriately summarizes the signature requirements for electronic filings (37 C.F.R. § 2.193(c) and TBMP §§ 106.02, 106.02(b)–(e)), including the use of slash‐enclosed electronic signatures and identification of the signatory’s name and title. It also correctly notes the rule regarding defective signatures (37 C.F.R. § 2.119(e) and TBMP § 106.02). The citations align with the source paragraphs. 

==== FINAL VERIFIED ANSWER ====
Verification: PASSED | Confidence: high

Answer:
A motion to compel discovery must be filed electronically with the Trademark Trial

検証ステップでは、特定の規制を参照し、回答の正確性と引用の適切な使用の両方を体系的にチェックする、整理された構造化された評価が生成されます。単に「正しい」と言うのではなく、なぜその回答が正しいのかを正確に説明することで有用なコンテキストを提供し、特定の引用とともにユーザーに回答を提示する際の確信を与えてくれます。

## 4. インフラストラクチャコスト

このエージェント型RAGアプローチのコスト構造を詳しく見てみましょう：

### 固定費と変動費の推定

* **推定固定費（一回限り）：**  
  * **従来のRAG:** 約$0.43（埋め込み + メタデータ生成）
  * **エージェント型RAG:** $0.00（前処理不要）


* **推定変動費（クエリあたり）：**  
  * **ルーターモデル（`gpt-4.1-mini`）：**  
    * 初期ルーティング（20チャンク）: 約$0.10  
    * 2つの再帰レベル: 約$0.20
  * **合成（`gpt-4.1`）:** 約$0.05
  * **検証（`o4-mini`）:** 約$0.01
  * **クエリあたりの合計:** 約$0.36

クエリあたりのコストは従来のRAGより高いですが、このアプローチは以下を提供します：
- 新しいドキュメントでの即座の結果
- より正確な引用
- 言い換えや概念的な質問のより良い処理
- インフラストラクチャ保守のオーバーヘッドなし

コストは以下により最適化できます：
- 一般的なクエリの結果をキャッシュ
- モデル呼び出しでの最大トークン数の制限
- ドキュメントを最初に事前フィルタリングするハイブリッドアプローチの使用

## 5. 従来のRAGとの利点とトレードオフ

### 利点
- **ゼロ取り込み遅延**: 前処理なしで新しいドキュメントから即座に質問に回答。
- **動的ナビゲーション**: 有望なセクションに焦点を当てることで、人間の読書パターンを模倣。
- **セクション間推論**: モデルは独立したチャンク検索では見逃される可能性のあるドキュメントセクション間の接続を見つけることができ、生成された回答の精度を向上させ、検索パイプラインの最適化時間を節約する可能性があります。

### トレードオフ
- **クエリあたりのコストが高い**: 埋め込みベースの検索と比較して、各質問により多くの計算が必要。
- **遅延の増加**: 階層ナビゲーションは単純なベクトル検索よりも処理に時間がかかる。
- **スケーラビリティの制限**: 前処理がより効率的になる非常に大きなドキュメントコレクションでは苦労する可能性。

## 6. 今後のステップ

採用したアプローチに対していくつかの修正を行うことができます：
- **ナレッジグラフの生成**: GPT 4.1-miniの大きなコンテキストウィンドウを使用して詳細なナレッジグラフを反復的に生成し、その後GPT 4.1がこのグラフを横断して質問に答えることができます。この方法では、質問に関係なく、ドキュメントを一度だけ「取り込む」必要があります。
- **改良されたスクラッチパッドツール**: スクラッチパッドツールには、過去のメモリの編集や削除などのより多くの選択肢を与えることができます。これにより、モデルは手元の質問に最も関連するものを選択できるようになります。
- **深度の調整**: 階層ナビゲーションの深度を調整して、コストとパフォーマンスの適切なバランスを見つけることができます。特定のユースケースでは文レベルの引用（法的文書など）が必要ですが、他のケースでは段落レベルの引用（ニュース記事など）のみが必要な場合があります。

## 7. 要点

1. **コンテキストウィンドウは超能力**: 100万トークンのコンテキストウィンドウにより、ドキュメントをその場でナビゲートすることが可能になります。
2. **階層アプローチは人間の読書を模倣**: エージェント型ルーティングは、関連セクションを求めてドキュメントを流し読みする人間のように機能します。
3. **スクラッチパッドは多段階推論を可能にする**: 推論記録の維持によりナビゲーション品質が向上します。
4. **高速実装、データベース不要**: システム全体をAPIコールのみで構築でき、インフラストラクチャは不要です。
5. **検証により信頼性が向上**: LLM-as-judgeパターンがユーザーに届く前にエラーをキャッチします。

================================================================================

## 3B. ユースケース: 製薬R&DのためのAI共同研究者
![AI Co-Scientist for Pharma R&D](../../../images/3B_reasoning_task_card.png)

このセクションでは、製薬R&Dにおける実験設計を加速するための「共同研究者」として機能するAIシステムの構築方法を詳述し、特定の制約下での薬物合成プロセスの最適化に焦点を当てます。

## 🗂️ TL;DRマトリックス

この表は、この特定のAI共同研究者実装のコア技術選択とその根拠をまとめています。

| レイヤー              | 選択                                                                  | 有用性                                                                                           |
| :----------------- | :------------------------------------------------------------------------ | :------------------------------------------------------------------------------------------------------- |
| **アイデア創出**       | `o4-mini`（並列ロールプレイングエージェント）                                  | 多様な仮説とプロトコルを迅速かつコスト効率的に生成；ロールプレイングが創造性を向上。 |
| **グラウンディング**      | 外部ツール呼び出し（`chem_lookup`、`cost_estimator`、`outcome_db`など） | 計画が実世界のデータ（化学的性質、コスト、過去の結果）に基づくことを保証。                   |
| **ランキング**        | `o4-mini`（ペアワイズトーナメント比較）                                | 単純なスコアリングを超えた微妙な評価；有望な候補を効率的に選択。                      |
| **批評/合成** | `o3`（深いレビューと合成）                                            | 厳密なシニアレベルの分析を提供し、リスクを特定し、科学的妥当性を保証。             |
| **安全性（オプション）**  | `gpt-4.1-mini`（対象チェック）                                       | 人間への引き渡し前に専門的な安全性レビューの追加レイヤーを提供。                                   |
| **学習**       | `o3` + Code Interpreter（結果分析 → DB）                            | 実験結果を体系的に捕捉し、時間をかけて継続的改善を可能にする。                |
| **コア技術** | マルチエージェント協調とエスカレーション                                    | 複雑な多段階推論タスクに対して異なるモデルの強み（速度対深度）を活用。      |

*注：モデル識別子は2025年4月時点で正確、変更される可能性があります。*

## 1. シナリオスナップショット

* **問題領域:** 製薬R&Dにおける複雑な実験手順の最適化、例えば厳格な制約を遵守しながら新薬化合物（「XYZ-13」）の合成収率を改善。
* **ユーザー:** 薬物発見と開発に関わる研究科学者と実験技術者。
* **典型的な要求:**
    1. 承認された試薬を使用して$15k以下で、異なる触媒をテストしてXYZ-13収率を≥15%増加させる3つの異なるプロトコルを提案。
    2. 過去の加熱問題により60°C以下でXYZ-13収率を最適化し、予算内で異なる承認溶媒を探索するプロトコルを提案。
    3. 2つのXYZ-13収率戦略を設計（≥15%を目指す）：a. $15k予算内で潜在収率を最大化するもの、b. $10k以下でコストを優先するもの。
* **制約:**
    * **予算:** 定義された財政限度内での運営（例：実験シリーズあたり$15,000）。
    * **規制/安全性:** 事前承認された化学物質/試薬のみを使用し、安全プロトコルを厳格に遵守。
    * **人間の監視:** 最終実験計画は実行前に人間の専門家によるレビューと検証が必要。

> 従来、このような実験の最適化には、手動計画、文献レビュー、反復的なベンチワーク、分析の数週間が必要でした。このAI共同研究者アプローチは、仮説生成、プロトコル設計、予備評価を自動化することで、サイクル時間を劇的に短縮し、科学者がより高レベルの戦略と最終検証に集中できるようにすることを目指しています。これにより、科学者の役割は計画ステップの手動実行から、AIとの専門的監視と協力へと移行します。

## 2. アーキテクチャ（マルチエージェント推論）

システムは高性能な科学チームを模倣するマルチエージェントアーキテクチャを採用しています。異なるAIコンポーネントが専門的な役割（アイデア創出、批評、結果からの学習など）で行動し、様々なモデルとツールを使用してワークフローを実行するために協力します。

![AI Co-Scientist Architecture](../../../images/3B_coscientist_architecture.png)

### 2.1. **科学者の入力と制約:** 
プロセスは科学者が目標、対象化合物、制約を定義することから始まります。

In [11]:
from openai import OpenAI
from agent_utils import Context, call_openai, log_json

# Example Initial Input
user_input = {
    "compound": "XYZ-13",
    "goal": "Improve synthesis yield by 15%",
    "budget": 15000,
    "time_h": 48,
    "previous": "Prior attempts failed at high temp; explore potential catalyst effects."
}
ctx = Context(client=OpenAI(), **user_input)

### 2.2. **アイデア創出（`o4-mini` + ツール）：**
異なる役割（例：`Hypothesis Agent`、`Protocol Agent`、`Resource Agent`）を与えられた複数の`o4-mini`インスタンスが、実験計画を並行して生成します。異なるペルソナを割り当てることで、多様な視点を促し、アイデア創出段階において問題の異なる側面を同時にカバーします。

In [12]:
ROLE_FOCUS = {
    # Hypothesis Agent Prompt
    "hypothesis_agent": """You are a pharmaceutical hypothesis specialist. 
        Focus exclusively on analyzing the compound structure and research goals to generate testable hypotheses. 
        Consider mechanism of action, binding affinity predictions, and potential off-target effects.""",

    # Protocol Agent Prompt
    "protocol_agent"  : """You are a laboratory protocol specialist. 
        Design experimental procedures that will effectively test the provided hypothesis. 
        Focus on experimental conditions, controls, and measurement techniques.""",

    # Resource Agent Prompt
    "resource_agent"  : """You are a laboratory resource optimization specialist. 
        Review the proposed protocol and optimize for efficiency. 
        Identify opportunities to reduce reagent use, equipment time, and overall costs while maintaining scientific validity.""",
}

# Create a structured prompt template for ideation
IDEATION_PROMPT = """You are a pharmaceutical {role} specialist. Your goal is to {goal} for compound {compound}.
Constraints:
- Budget: ${budget}
- Approved reagents only
- Complete within {time_h} hours
- Previous attempts: {previous}
Respond with structured JSON describing your protocol."""

In [13]:
import json, logging
from pathlib import Path
from typing import Dict, List, Any, Optional
from dataclasses import asdict
from functools import partial

MODEL_IDEATE   = "o4-mini-2025-04-16"  # o4-mini model for ideation - balances speed and quality

# Configure logging to help with tracking experiment progress and debugging
logging.basicConfig(level=logging.INFO, format="%(message)s")
logging.info(f"Run‑id {ctx.run_id}  Compound: {ctx.compound}")
logging.info(f"Logs will be stored in: {Path('logs') / ctx.run_id}")

def ideation(ctx: Context):
    logging.info("Starting ideation phase...")
    ideas = []
    for role, focus in ROLE_FOCUS.items():
        logging.info(f"Running ideation agent ${role}")
        sys = IDEATION_PROMPT.format(role=role, focus=focus, **ctx.prompt_vars())
        usr = f"Design a protocol to {ctx.goal} within ${ctx.budget}."
        idea = call_openai(ctx.client, MODEL_IDEATE, sys, usr, ctx)
        ideas.append(idea)
    log_json("ideation_done", ideas, ctx)
    return ideas

Run‑id 9835f69c  Compound: XYZ-13
Logs will be stored in: logs/9835f69c


アイデア生成エージェントは、`literature_search`、`chem_lookup`（化学データベース）、`cost_estimator`、`outcome_db`（過去の実験結果）などの外部ツールを活用して、データに基づいた提案を行うことができます。外部ツールの使用を明示的に有効化し、モデルにその使用を促すことで、生成される計画が実現可能で、規則に準拠し、既存の知識に基づいたものになることが保証されます。モデルは、タスクに基づいていつ、どのツールを呼び出すかを決定します。

In [14]:
IDEATION_PROMPT += """\nUse the following tools as appropriate:
- Use the `list_available_chemicals` tool to get list of approved reagents.
- Use the `chem_lookup` tool to verify properties of reagents mentioned.
- Use the `cost_estimator` tool to calculate the approximate cost based on reagents and proposed steps.
- Check the `outcome_db` for relevant prior experiments with {compound}"""

ideas = ideation(ctx)
logging.info("Ideation complete!")

Starting ideation phase...
Running ideation agent $hypothesis_agent
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
(Tool) List available chemicals
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
(Tool) Outcome DB: XYZ-13, yield, 5
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
(Tool) Cost estimator: [{'name': 'Palladium chloride', 'amount': 0.05, 'unit': 'g'}, {'name': 'Triphenylphosphine', 'amount': 0.1, 'unit': 'g'}, {'name': 'Potassium carbonate', 'amount': 1, 'unit': 'g'}, {'name': 'Dimethylformamide', 'amount': 50, 'unit': 'mL'}, {'name': 'Toluene', 'amount': 50, 'unit': 'mL'}, {'name': 'Sodium borohydride', 'amount': 0.1, 'unit': 'g'}, {'name': 'Triethylamine', 'amount': 0.5, 'unit': 'mL'}], ['round-bottom flask', 'magnetic stirrer', 'reflux condenser'], 36
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Running ideation agent $protocol_agent
HTTP Requ

これらのツールは`agent_utils.py`で定義されています。このソリューションでは、ツール呼び出しは`tools.py`でモックされています。実際の使用例では、これらのツールは実際のAPIを呼び出すことになります。

### 2.3. **トーナメントランキング（`o4-mini` / `o3`）：**
生成されたプロトコルは、期待される効果、実現可能性、コスト、新規性などの基準に基づいてペアワイズで比較されます。モデルにプロトコルを単独で評価させるのではなく、一度に2つのプロトコルを提供し、特定の基準に対する直接比較を求めることで、より信頼性の高い相対的なランキングが得られることが多いです。

このEloスタイルのランキングにより、より深いレビューに値する最も有望な候補を特定します。

In [15]:
TOURNAMENT_PROMPT = """
Protocol A: [details...]
Protocol B: [details...]

Compare Protocol A and Protocol B for synthesizing {compound} aimed at {goal}. Score them on:
1. Likelihood of achieving ≥ 15% yield increase.
2. Practical feasibility (reagents, time).
3. Estimated cost-efficiency (use tool if needed).
4. Scientific novelty/risk.

Return JSON {{\"winner\": \"A\"|\"B\", \"justification\": \"...\"}}."""

# This is a mock tourname implementation that only compares the first two protocols
# A real implementation would compare pairs in a tournament bracket style
def tournament(protocols: List[Dict[str, Any]], ctx: Context):
    logging.info("Starting tournament phase...")
    if len(protocols) == 1:
        return protocols[:1]
    a, b = protocols[0], protocols[1]
    sys = TOURNAMENT_PROMPT.format(**ctx.prompt_vars())
    usr = json.dumps({"A": a, "B": b}, indent=2)
    res = call_openai(ctx.client, MODEL_IDEATE, sys, usr, ctx)
    winner = a if res.get("winner", "A").upper() == "A" else b
    log_json("tournament", res, ctx)
    return [winner]

top_proto = tournament(ideas, ctx)[0]
logging.info("Tournament winner picked!")

Starting tournament phase...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Tournament winner picked!


> 初期の実験では、モデルに1-10のスケールでプロトコルを評価するよう求めると、スコアの圧縮により一貫性のない結果が生じることがわかりました。トーナメント方式はこの問題を解決し、より信頼性の高い相対的な判断を強制することができました。これは人間の専門家の行動を反映しています — 科学者は絶対的なスコアを割り当てるよりも、2つの選択肢を直接比較する方が容易であることが多いのです。

### 2.4. **深い批評と統合 (`o3`):** 
上位にランクされたプロトコルは、厳密なレビューのために`o3`に渡されます。`o3`はシニア科学者のように振る舞い、科学的妥当性、方法論、安全性、予算遵守を評価し、改善を提案したり、最終的な洗練されたプロトコルを統合したりします。また、検証のためにツールを呼び出すこともあります。

In [16]:
# Deep critique phase using a more powerful model for rigorous review
CRITIQUE_PROMPT = """You are a senior researcher reviewing a proposed synthesis protocol 
for {compound} aiming for {goal}, budget ${budget} using approved reagents. Review the protocol below rigorously:
1. Identify scientific flaws or methodological weaknesses.
2. Assess safety risks and budget compliance (use `cost_estimator` tool if needed).
3. Check for consistency with prior `outcome_db` results if relevant.
4. Suggest concrete improvements or rewrite sections if necessary.
5. Provide a final go/no-go recommendation.

Return JSON {{\"revised_protocol\": ..., \"critique\": \"...\", \"recommendation\": \"go|no-go\"}}.

Protocol to Review:
[Protocol details...]
"""

MODEL_CRITIQUE = "o3-2025-04-16"  # o3 model for deep critique

def critique(protocol: Dict[str, Any], ctx: Context):
    logging.info("Starting critique phase...")
    sys = CRITIQUE_PROMPT.format(**ctx.prompt_vars())
    usr = json.dumps(protocol, indent=2)
    crit = call_openai(ctx.client, MODEL_CRITIQUE, sys, usr, ctx)
    log_json("critique", crit, ctx)
    return crit.get("revised_protocol", protocol)

critiqued = critique(top_proto, ctx)
logging.info("Deep critique completed!")

Starting critique phase...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
(Tool) Cost estimator: [{'name': 'Palladium chloride', 'amount': 0.0045, 'unit': 'g'}, {'name': 'Triphenylphosphine', 'amount': 0.013, 'unit': 'g'}, {'name': 'Sodium borohydride', 'amount': 0.0038, 'unit': 'g'}, {'name': 'Potassium carbonate', 'amount': 0.14, 'unit': 'g'}, {'name': 'Triethylamine', 'amount': 0.07, 'unit': 'mL'}, {'name': 'Dimethylformamide', 'amount': 2, 'unit': 'mL'}, {'name': 'Toluene', 'amount': 5, 'unit': 'mL'}], ['100 mL round-bottom flask', 'magnetic stirrer', 'reflux condenser', 'inert gas line'], 24
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
(Tool) Outcome DB: XYZ-13, None, 5
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Deep critique completed!


> 私たちは意図的に、異なるモデルとペルソナを使用してアイデア創出と批評を分離しています。同じモデルが自分の作業を生成し、同時に批評することは、客観的な評価よりもむしろ自己正当化につながることが多いのです。「シニアサイエンティスト」として機能するo3モデルは、アイデア創出段階でo4-miniが見逃した方法論的な弱点を一貫して特定しました。

### 2.5. **（オプション）安全性チェック：**
`gpt-4.1-mini`などの専門モデルが、特定の安全性に関する懸念（例：危険な試薬の組み合わせ）について最終チェックを実行できます。

In [17]:
# Optional safety check using a targeted model
SAFETY_PROMPT = """You are a lab‑safety specialist. 
Identify hazards, unsafe conditions, or compliance issues in this protocol for {compound}. 
Use `chem_lookup` tool if needed. Return JSON assessment."""

MODEL_SAFETY   = "gpt-4.1-mini-2025-04-14"  # gpt-4.1-mini model for safety checks - optimized for instruction following

def safety(protocol: Dict[str, Any], ctx: Context):
    logging.info("Starting safety assessment...")
    sys = SAFETY_PROMPT.format(**ctx.prompt_vars())
    usr = json.dumps(protocol, indent=2)
    assessment = call_openai(ctx.client, MODEL_SAFETY, sys, usr, ctx)
    log_json("safety", assessment, ctx)
    return {"protocol": protocol, "safety": assessment}

secured = safety(critiqued, ctx)
logging.info("Safety check completed!")

Starting safety assessment...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
(Tool) Chemical lookup: Palladium chloride, None
(Tool) Chemical lookup: Triphenylphosphine, None
(Tool) Chemical lookup: Sodium borohydride, None
(Tool) Chemical lookup: Potassium carbonate, None
(Tool) Chemical lookup: Dimethylformamide, None
(Tool) Chemical lookup: Toluene, None
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Safety check completed!


### 2.6. **人間によるレビュー：**
AIが生成した最終計画は、検証、潜在的な編集、および最終承認のために、インターフェースを通じて人間の科学者に提示されます。

In [18]:
def human_review(safety_package: Dict[str, Any], ctx: Context):
    logging.info("Awaiting human review...")
    protocol = safety_package["protocol"]
    safety_assessment = safety_package["safety"]
    
    print(f"\n=== PROTOCOL FOR REVIEW: {ctx.compound} - {ctx.goal} ===")
    print(f"DETAILS: {json.dumps(protocol, indent=2)}")
    print(f"SAFETY: {json.dumps(safety_assessment, indent=2)}")
    
    while True:
        approval = input("\nApprove for execution? (yes/no): ").lower()
        if approval in ['yes', 'y', 'no', 'n']:
            approved = approval in ['yes', 'y']
            logging.info(f"Protocol {'approved' if approved else 'rejected'}")
            return {"protocol": protocol, "approved": approved}
        print("Please enter 'yes' or 'no'")

human_decision = human_review(secured, ctx)

Awaiting human review...



=== PROTOCOL FOR REVIEW: XYZ-13 - Improve synthesis yield by 15% ===
DETAILS: {
  "protocol_title": "Optimised In-Situ Pd(0)/PPh3 Coupling for XYZ-13 \u2013 Target \u2265 72 % Yield",
  "key_changes_vs_original": [
    "Catalyst loading reduced from 5 mol % to 2 mol % Pd to cut cost and metal contamination without loss of activity.",
    "Reaction run at 0.10 M substrate concentration (12 mL solvent total) instead of 50 mL; higher effective collision frequency boosts conversion and reduces waste.",
    "Single solvent system (toluene/DMF 4:1) avoids phase separation and simplifies work-up.",
    "Redundant triethylamine removed; K2CO3 (2.5 eq) provides sufficient basicity.",
    "Reaction temperature raised slightly to 80 \u00b0C (still below side-reaction threshold found in exp-001) and time shortened to 24 h with in-process HPLC check at 6 h intervals.",
    "Work-up switched from large silica column to two-step: (a) aqueous EDTA wash to strip Pd, (b) recrystallisation from EtOAc/he

Protocol approved


### 2.7. **実行と学習 (`o3` + Code Interpreter):** 
人間が承認すると、計画は実験室での実行に送られます。実験室での実行後、結果はシステムにフィードバックされます。`o3`と`Code Interpreter`を組み合わせることで、データを分析し、洞察を生成し、構造化された成果（プロトコル、パラメータ、結果、洞察）をデータベース（`Outcome DB`）に保存します。このデータベースは将来の発想サイクルに情報を提供し、学習ループを作り出します。

In [19]:
# Simulating execution and analyzing results
ANALYSIS_PROMPT = """You are a data analyst. 
Did the experiment achieve {goal}?  Analyse factors, suggest improvements, and return structured JSON.
"""

def execute_and_analyse(pkt: Dict[str, Any], ctx: Context):
    logging.info("Starting mock execution and analysis...")
    # These are mock results for a lab experiment
    mock_results = {
        "yield_improvement": 12.5,
        "success": False,
        "actual_cost": ctx.budget * 0.85,
        "notes": "Mock execution"
    }
    sys = ANALYSIS_PROMPT.format(**ctx.prompt_vars())
    usr = json.dumps({"protocol": pkt, "results": mock_results}, indent=2)
    analysis = call_openai(ctx.client, MODEL_CRITIQUE, sys, usr, ctx)
    log_json("analysis", analysis, ctx)
    return analysis

# Only proceed to execution if approved by the human reviewer
if human_decision["approved"]:
    summary = execute_and_analyse(human_decision, ctx)
    logging.info("Analysis complete")
else:
    logging.info("Protocol rejected by human reviewer - execution skipped")
    summary = None

Path("output").mkdir(exist_ok=True)
out_path = Path("output") / f"{ctx.run_id}_summary.json"
out_path.write_text(json.dumps(summary, indent=2))
print(f"\n🎉 Completed. Summary written to {out_path}")

Starting mock execution and analysis...
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
(Tool) Literature search: Pd(0) PPh3 coupling yield optimization EDTA work-up recrystallization losses, None, 3
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
(Tool) Outcome DB: XYZ-13, yield, 5
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Analysis complete



🎉 Completed. Summary written to output/9835f69c_summary.json


## 3. モデルプレイブック

`o4-mini`と`o3`の選択は、タスクの複雑さと必要な深度によって決まります。その他のタスクでは、`gpt-4.1-mini`がコストとパフォーマンスのバランスを提供し、より高い能力やニュアンスが必要な場合は、より強力な`gpt4.1`が推奨されます。

| タスク               | 開始モデル     | アップグレード条件...                                            | エスカレーション先  | 根拠                                                                                    |
| :----------------- | :------------- | :--------------------------------------------------------- | :----------- | :------------------------------------------------------------------------------------------- |
| アイデア創出とプロトコル生成 | `o4-mini` | 仮説に深度や複雑な化学合成に必要な創造性が不足している場合 | `o3` | `o4-mini`は多様なプロトコルを費用効率よく迅速に生成します。より微妙なアプローチが必要な場合、`o3`がより深い科学的推論を提供します。 |
| プロトコルランキング | `o4-mini` | 比較により深い科学的評価や多要素のトレードオフが必要な場合 | `o3` | `o4-mini`によるトーナメント形式のランキングで有望な候補を効率的に特定します。微妙な科学的妥当性の評価が必要な場合にエスカレートします。 |
| 深い批評と統合 | `o3` | N/A - この重要なタスクには既に最も高性能なモデルを使用 | N/A | `o3`は厳密な科学的レビュー、方法論的欠陥の特定、複雑なプロトコル間での改善の統合に優れています。このタスクは本質的に深い推論を必要とします。 |
| 安全性評価 | `gpt-4.1-mini` | ドメイン固有の危険性により高い精度や専門知識が必要な場合 | `gpt-4.1` | `gpt-4.1-mini`は標準的な安全性チェックにおいてコストとパフォーマンスの良いバランスを提供します。複雑な安全性リスクに対してより高い精度やより微妙な推論が必要な場合は`gpt4.1`にエスカレートします。 |

**重要な洞察:**
> このユースケースは強力なパターンを例示しています：より高速で安価なモデル（`o4-mini`）を幅広い範囲と初期フィルタリングに使用し、その後より強力なモデル（`o3`）に深度、批判的レビュー、統合のためにエスカレートします。この階層化されたアプローチは、創造性/速度と厳密性/精度の両方を最適化しながら、計算コストを効果的に管理します。ツールとの統合は、AIの推論を検証可能な実世界のデータに基づかせるために不可欠です。

## 4. デプロイメントノート

AI Co-Scientistをプロトタイプから実験室での使用に移行するには、慎重な計画が必要です。

* **コスト管理:**
    * `o4-mini`アイデア創出エージェントの数、`o3`批評の深度、またはオプションチェックの使用を調整して結果品質とコストおよびレイテンシのバランスを取る設定可能な「モード」（`Fast`、`Standard`、`Thorough`など）を実装します。
    * 段階ごと（アイデア創出、ランキング、批評）およびツール呼び出しごとのトークン使用量を追跡して、きめ細かいコスト監視を行います。
* **観測可能性:**
    * 各ステップの入力、出力、モデル選択、ツール呼び出し/応答、レイテンシ、トークン数をログに記録します。
    * トーナメントランキングのパフォーマンスと`o3`批評の影響（計画が大幅に変更または拒否される頻度など）を監視します。
    * ユーザーインタラクションを追跡：人間の科学者によって承認、編集、または拒否された計画を記録します。
* **安全性とコンプライアンス:**
    * 複数の安全層を実装：プロンプトでの制約、ツールベースのチェック（`chem_lookup`による試薬互換性など）、オプションの専用モデルチェック（`gpt-4.1-mini`）、自動フィルター（既知の危険な組み合わせなど）、必須の人間レビュー。
    * ツールエンドポイント（内部データベースなど）がセキュリティ要件を満たすことを確保します。
* **展開戦略:** 
    * 過去の実験の遡及分析から始め、シャドウモード（AIが人間の計画者と並行して計画を提案）に移行し、その後限定的なライブユースケースで密接な監視を行い、より広範な採用の前に実施します。

## 5. 重要なポイント

1. **モデルペアリングが相乗効果を生む**: `o4-mini`がより広範囲を迅速にカバーし、`o3`が精度と深度をもたらします。
2. **ツール統合が推論を現実に基づかせる**: 化学コストや安全制約などの実世界のデータが意思決定に情報を提供します。
3. **人間の科学者が中心的存在**: システムは専門家を置き換えるのではなく、雑務を取り除くことで専門家を支援します。

## 6. 有用なクックブックとリソース

AI Co-Scientistシステムの設計と実装を補完する厳選されたリソースは以下の通りです：

- **[Orchestrating Agents: Routines and Handoffs](https://cookbook.openai.com/examples/orchestrating_agents)** ルーチンとハンドオフによるマルチエージェントワークフローの構造化、アイデア創出→ランキング→批評パイプラインに関連。

- **[GPT-4.1 Prompting Guide](https://cookbook.openai.com/examples/gpt4-1_prompting_guide)** 批評と安全性レビューの精度向上のための高度なプロンプティング、ツール使用、タスク分解。

- **[Structured Outputs for Multi-Agent Systems](https://cookbook.openai.com/examples/structured_outputs_multi_agent)** エージェント間の相互運用性のためのスキーマ検証による一貫したJSON出力の強制。

- **[Agents - OpenAI API](https://platform.openai.com/docs/guides/agents)**  
  OpenAIツールを使用したマルチエージェントシステム構築の包括的ガイド、このシステムのアーキテクチャの基盤となるオーケストレーション、ツール使用、ベストプラクティスをカバー。

================================================================================

## 3C. ユースケース: 保険請求処理

![](../../../images/3C_insurance_task_card.png)

多くの企業は手書きフォームのデジタル化というタスクに直面しています。このセクションでは、OpenAIを使用して手書きの保険フォームをデジタル化し、検証する方法を実演します。これは保険業界でよくある問題ですが、同じ技術は税務フォーム、請求書など、他の様々な業界やフォームにも適用できます。

## 🗂️ TL;DR マトリックス

この表は、保険ユースケースを対象とした特定のOCR実装における中核技術選択とその根拠をまとめています。

| レイヤー | 選択 | 有用性 |
| :---- | :---- | :---- |
| JSON出力 | Pydanticによる構造化出力 | フォーマットの指定が簡単、`JSONモード`よりもスキーマに準拠 |
| OCRと視覚 | `gpt-4.1` | 強力なOCRと視覚機能、構造化出力 |
| 推論 | `o4-mini` | 手頃な価格でありながら高性能な推論、関数呼び出し利用可能 |
| フォーム検証 | カスタム関数呼び出し | カスタムまたは内部データベースとの相互作用を提供可能 |

\*注：価格とモデル識別子は2025年4月時点で正確、変更される可能性があります。

## 1. シナリオスナップショット

* **ユーザー:** 対象ユーザーは手書きフォームからデータを取り込む必要がある保険サービスおよび運用チームです。
* **典型的な要求:** 各フォームは異なる必要な構造と、抽出が必要な異なるフィールドを持ちます。
* **制約:**  
  * **精度:** データが正確で完全であることを確保するために高い精度が必要です。
  * **不確実性:** システムは、欠損データ、曖昧なデータ、同じフィールドの異なる形式など、データの不確実性を処理する必要があります。モデルが不確実性を解決できない場合、システムには人間のレビューを要求するメカニズムが必要です。
  * **パフォーマンスとコスト:** システムのレイテンシは重要ではありませんが、コストを抑制しながら高い精度が必要です。1000ページ処理あたり20ドル以下のコスト目標を設定します。

## 2. アーキテクチャ

ソリューションの高レベルな基本アーキテクチャを以下に示します。

![](../../../images/3C_insurance_architecture.png)

このタスクは複雑で、視覚、関数呼び出し、推論、構造化出力など、幅広いモデル機能を必要とします。`o3`はこれらすべてを一度に実行できますが、実験中に`o4-mini`だけでは必要なパフォーマンスを達成するのに十分でないことがわかりました。`o3`の相対的に高いコストのため、代わりに2段階アプローチを選択しました。

1. 第1段階はGPT 4.1の視覚機能を使用して実行されます。この段階は最大精度でテキストを抽出するように最適化されており、推論段階のために不確実性を残し、ページに見えない仮定は行いません。第1段階でOCRを行うことで、推論モデルが画像から直接作業する必要がなくなります。これは、推論モデルが実行しなければならない他のすべてのタスクを考慮すると困難な場合があります。

2. 第2段階は`o4-mini`の推論能力を活用します。`o4-mini`を使用してOCRの精度を検証し、データを構造化形式に抽出します。重要なことは、o4-miniが二次品質ゲートとして機能することを期待していることです -- この段階でOCRが不完全な場合、o4-miniを使用して元の結果を改良し検証できます。

これがどのように機能するかを具体的に実演するために、保険フォームのサンプル画像を見てみましょう。

![](../../../images/3C_insurance_form.png)

フォーム自体はかなり単純ですが、従来のOCRシステムが正しく記入するのが困難な欠損データと曖昧な情報があります。まず、郵便番号と郡が省略されていることに注目してください。次に、ユーザーのメールアドレスが曖昧です -- `jsmith1@gmail.com`または`jsmithl@gmail.com`の可能性があります。以下のセクションでは、適切に設計されたソリューションがこれらの曖昧さを処理し、正しいフォーム結果を返す方法を説明します。

**環境設定とライブラリコード:**

サンプルコードをより明確にするために、環境設定（`pip install`コマンドなど）とライブラリ関数を別のコードブロックに分離しました。これにより、ソリューションの各ステップで関連するロジックのみに焦点を当てやすくなります。

In [6]:
# Install Python requirements
%pip install -qU pydantic "openai>=1.76.0"

# All imports
import os
import json

from pydantic import BaseModel

# Create the OpenAI client
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "sk-dummykey"))

Note: you may need to restart the kernel to use updated packages.


In [7]:
def run_conversation_loop(
    client,
    messages,
    tools,
    tool_handlers,
    response_format,
    model,
):
    """Run the OpenAI response completion loop, handling function calls via tool_handlers until parsing final response."""
    summaries = []
    while True:
        print(
            f"Requesting completion from model '{model}' (messages={len(messages)})"
        )
        response = client.responses.parse(
            model=model,
            input=messages,
            tools=tools,
            text_format=response_format,
            reasoning={"summary": "auto"},
        )
        summaries.append(response.output[0].summary)

        if not response.output_parsed:
            print("Assistant requested tool calls, resolving ...")

            reasoning_msg, tool_call = response.output
            messages.append(reasoning_msg)
            messages.append({
                "id": tool_call.id,
                "call_id": tool_call.call_id,
                "type": tool_call.type,
                "name": tool_call.name,
                "arguments": tool_call.arguments,
            })

            if tool_call.name in tool_handlers:
                try:
                    args = json.loads(tool_call.arguments)
                except Exception as exc:
                    print(
                        "Failed to parse %s arguments: %s", tool_call.name, exc
                    )
                    args = {}
                result = tool_handlers[tool_call.name](**args)
                messages.append(
                    {
                        "type": "function_call_output",
                        "call_id": tool_call.call_id,
                        "output": str(result),
                    }
                )
                print(f"Tool call {tool_call.name} complete, result: {str(result)}")
            else:
                print("Unhandled function call: %s", tool_call.name)

        if response.output_parsed is not None:
            print("Received parsed result from model")
            return response, summaries

**フロー説明：ステージ1**

1. **画像：** ユーザーのスマートフォンから撮影されたフォームの画像がモデルに渡されます。OpenAIのモデルは様々な画像形式を受け入れることができますが、通常はテキストを鮮明に保ち、アーティファクトを減らすためにPNG形式を使用します。この例では、公開されているコンテンツURLから画像をモデルに渡します。本番環境では、おそらく自社のクラウドストレージバケットにホストされた画像への署名付きURLとして画像を渡すことになるでしょう。

2. **構造化出力スキーマ：** 出力データの構造を設定するPydanticモデルを定義します。このモデルには、フォームから抽出する必要があるすべてのフィールドと、各フィールドの適切な型が含まれています。私たちのモデルは複数のサブコンポーネントに分かれており、それぞれがPydanticモデル自体であり、親モデルから参照されています。

In [8]:
class PersonContact(BaseModel):
    name: str
    home_phone: str
    work_phone: str
    cell_phone: str
    email: str

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip: str
    county: str

class DwellingDetails(BaseModel):
    coverage_a_limit: str
    companion_policy_expiration_date: str
    occupancy_of_dwelling: str
    type_of_policy: str
    unrepaired_structural_damage: bool
    construction_type: str
    roof_type: str
    foundation_type: str
    has_post_and_pier_or_post_and_beam_foundation: bool
    cripple_walls: bool
    number_of_stories: str
    living_space_over_garage: bool
    number_of_chimneys: str
    square_footage: str
    year_of_construction: str
    anchored_to_foundation: bool
    water_heater_secured: bool

class InsuranceFormData(BaseModel):
    applicant: PersonContact
    co_applicant: PersonContact
    risk_address: Address
    mailing_address_if_different_than_risk_address: Address
    participating_insurer: str
    companion_policy_number: str
    dwelling_details: DwellingDetails
    effective_date: str
    expiration_date: str

3. **OCRの実行:** GPT-4.1のビジョン機能を使用して、パイプラインの第一段階を実行し、構造化された形式で文書からテキストを抽出します。この初期段階では、不確実性を第二段階に渡しながら高い精度を達成することを目的としています。私たちのプロンプトは、推測による入力を避け、代わりに可能な限り正確に詳細を記入するようモデルに明示的に指示しています。画像入力については、画像入力の詳細レベルを`auto`に設定して、画像に適した詳細レベルを推測させます。実験では`auto`がうまく機能することがわかりましたが、OCR処理で品質に問題が見られる場合は`high`の使用を検討してください。

In [9]:
OCR_PROMPT = """You are a helpful assistant who excels at processing insurance forms.

You will be given an image of a hand-filled insurance form. Your job is to OCR the data into the given structured format.
Fill out the fields as exactly as possible. If a written character could possibly be ambiguous (i.e. l or 1, o or 0), include all possiblities in the field separated by "OR", especially for email addresses.
"""

user_content = [
    {"type": "input_text", "text": "Here is a photo of the form filled out by the user:"},
    {
        "type": "input_image",
        "image_url": "https://drive.usercontent.google.com/download?id=1-tZ526AW3mX1qthvgi8spaaxxeqFG5_6",
        "detail": "auto",
    },
]

messages = [
    {"role": "system", "content": OCR_PROMPT},
    {"role": "user", "content": user_content},
]

response = client.responses.parse(
    model="gpt-4.1-2025-04-14",
    input=messages,
    text_format=InsuranceFormData,
    # Set temp to 0 for reproducibility
    temperature=0,
)

s1_json_results = json.dumps(json.loads(response.output_parsed.model_dump_json()), indent=2)
print(s1_json_results)

{
  "applicant": {
    "name": "Smith, James L",
    "home_phone": "510 331 5555",
    "work_phone": "",
    "cell_phone": "510 212 5555",
    "email": "jsmithl@gmail.com OR jsmith1@gmail.com"
  },
  "co_applicant": {
    "name": "Roberts, Jesse T",
    "home_phone": "510 331 5555",
    "work_phone": "415 626 5555",
    "cell_phone": "",
    "email": "jrobertsjr@gmail.com"
  },
  "risk_address": {
    "street": "855 Brannan St",
    "city": "San Francisco",
    "state": "CA",
    "zip": "",
    "county": ""
  },
  "mailing_address_if_different_than_risk_address": {
    "street": "",
    "city": "",
    "state": "",
    "zip": "",
    "county": ""
  },
  "participating_insurer": "Acme Insurance Co",
  "companion_policy_number": "81265919",
  "dwelling_details": {
    "coverage_a_limit": "$900,000",
    "companion_policy_expiration_date": "5/31/27",
    "occupancy_of_dwelling": "Owner",
    "type_of_policy": "Homeowners",
    "unrepaired_structural_damage": false,
    "construction_type"

出力にいくつかのフィールドが欠けていることに注意してください。処理の次の段階では、OpenAIの推論モデルを活用して、可能な場合は欠けているフィールドを推測します。

**フロー説明：ステージ2**

1. **関数定義：** モデルが不確実性を解決するために使用できるカスタム関数のセットを定義します。この場合、メールアドレスが存在するかどうかをチェックしてメールアドレスを検証できる関数を定義します。これは、モデルが複数の可能な値の中から選択しなければならない曖昧なメールアドレスフィールドを解決するために使用できます。デフォルトでは、o4-miniはウェブ検索などの組み込みツールをサポートしており、この場合、郵便番号や不完全な住所を解決するために使用されます。

In [10]:
tools = [{
    "type": "function",
    "name": "validate_email",
    "description": "Check if an email address is valid and exists.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address to validate."
            }
        },
        "required": [
            "email"
        ],
        "additionalProperties": False
    }
},
{
    "type": "function",
    "name": "search_web",
    "description": "Perform a web search.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query to run through the search engine."
            }
        },
        "required": [
            "query"
        ],
        "additionalProperties": False
    }
}]

2. **プロンプト:** OCRによってテキストを抽出したことをモデルに説明し、欠損している、または曖昧なフィールドを埋めるために推論と関数呼び出しを実行するよう要求するプロンプトを提供します。

In [11]:
PROMPT = """You are a helpful assistant who excels at processing insurance forms.

You will be given a javascript representation of an OCR'd document. Consider at which fields are ambiguous reason about how to fill them in. Fill any missing fields that are possible to infer from existing data, or search the web. If you cannot fill a field, reason about why.

Use the tools provided if necessary to clarify the results. If the OCR system has provided two possibilities, do your best to definitely pick which option is correct.
"""

In [12]:
messages = [
    {"role": "system", "content": PROMPT},
    {"role": "user", "content": s1_json_results},
]

# For demonstration purposes, we'll hardcode the correct email answer.
def email_mock(*args, **kwargs):
    if kwargs["email"] == "jsmithl@gmail.com":
        return True
    return False

# Reasoning models like `o4-mini` will soon support built-in web search, but for now
# we demonstrate this capability using a simple mock function.
def web_mock(*args, **kwargs):
    if "855 Brannan" in kwargs["query"]:
        return "855 Brannan St, San Francisco, 94103, San Francisco County"
    
    return ""
    
tool_handlers = {"validate_email": email_mock, "search_web": web_mock}

response, summaries = run_conversation_loop(
    client=client,
    messages=messages,
    tools=tools,
    tool_handlers=tool_handlers,
    response_format=InsuranceFormData,
    model="o4-mini-2025-04-16",
)

print(json.dumps(json.loads(response.output_parsed.model_dump_json()), indent=2))

Requesting completion from model 'o4-mini-2025-04-16' (messages=2)
Assistant requested tool calls, resolving ...
Tool call validate_email complete, result: True
Requesting completion from model 'o4-mini-2025-04-16' (messages=5)
Assistant requested tool calls, resolving ...
Tool call validate_email complete, result: False
Requesting completion from model 'o4-mini-2025-04-16' (messages=8)
Received parsed result from model
{
  "applicant": {
    "name": "Smith, James L",
    "home_phone": "510 331 5555",
    "work_phone": "",
    "cell_phone": "510 212 5555",
    "email": "jsmithl@gmail.com"
  },
  "co_applicant": {
    "name": "Roberts, Jesse T",
    "home_phone": "510 331 5555",
    "work_phone": "415 626 5555",
    "cell_phone": "",
    "email": "jrobertsjr@gmail.com"
  },
  "risk_address": {
    "street": "855 Brannan St",
    "city": "San Francisco",
    "state": "CA",
    "zip": "94107",
    "county": "San Francisco"
  },
  "mailing_address_if_different_than_risk_address": {
    "st

メールアドレスが単一の値に絞り込まれ、郵便番号と郡が入力され、リスクアドレスを使用して郵送先住所が入力されていることがわかります。モデルはまた、構造化された形式（はい/いいえの質問にはbooleanなど適切な型を使用）で結果を返しており、下流システムで簡単に解析できます。

モデルを理解し、デバッグするために、モデルが生成した要約の思考連鎖推論を出力することもできます。これにより、一般的な失敗モード、モデルが不明確な点、または上流の詳細が間違っている点を明らかにするのに役立ちます。

このソリューションを開発している間、思考連鎖の要約により、間違った名前や型が付けられたスキーマ値がいくつか明らかになりました。

In [13]:
for summary in summaries:
    for response in summary:
        print(response.text + '\n')

**Determining insurance form details**

I have a JSON representation of a partially filled insurance form, and there are a few missing or ambiguous fields that I need to address.

For the email address, I see two options. I can validate which one is correct by checking both with the tool.

The risk address fields for zip code and county are empty. Based on the address "855 Brannan St, San Francisco, CA," I can determine the correct zip code is 94107, as that area corresponds to South Beach. Lastly, since the mailing address is empty, I assume it's the same as the risk address.

**Filling insurance form details**

I think it’s best to set the mailing address to be the same as the risk address or clarify that a blank one implies the same. Since it’s an explicit instruction to fill missing fields, I’ll fill in the mailing address with the risk address to avoid confusion.

All co-applicant fields are present, and dwelling details are complete. The effective and expiration dates are also pr

## 3. モデルと機能のプレイブック

適切なツールを選択することが最良の結果を得るための鍵です。一般的に、ニーズに合う最もシンプルなソリューションから始めて、より多くの機能が必要になったらアップグレードするのが良いアプローチです。

| タスク | 開始モデル | アップグレードのタイミング | 上位モデル | 理由 |
| :---- | :---- | :---- | :---- | :---- |
| OCR | `gpt-4.1` | 一目で理解が困難な複雑なフォーム | `o3` | `gpt-4.1`はほとんどのOCRタスクで高速かつコスト効率が良い。`o3`はフォーム構造について推論する能力を持つ。 |
| 結果の精緻化 | `o4-mini` | 詳細を推論するための複雑なロジック、多数の関数呼び出しが必要 | `o3` | 関数呼び出しと構造化出力の両方を含む、非常に長い推論チェーンに適している。 |

## 4. 評価指標

システムが正確に期待通りに動作していることを確認するために、重要な指標を追跡します。

### 重要な指標

* **OCR精度：** 文字単位および単語単位の精度。
* **推論フィールド率：** 既存データまたは関数呼び出しから正しく推論された未入力エントリの割合。
* **人間介入率：** 文書にUNKNOWNが含まれ、人間に参照する必要がある頻度。

フォームとその期待される応答のラベル付きホールドアウトセットを構築することを推奨します。このデータセットは期待される展開環境を代表するものであるべきです。システムの構築と評価に関する詳細情報については、[OpenAI evals](https://platform.openai.com/docs/guides/evals)ガイドを参照してください。

## 5. デプロイメント注意事項

プロトタイプから本番対応システムへの移行には、運用の詳細（LLMOps）への注意が必要です。

### コスト内訳

文書取り込みについては、高いレイテンシ許容度（つまり、夜間実行で問題ない）により、[バッチ価格](https://platform.openai.com/docs/guides/batch)が実行可能な選択肢であると仮定します。

#### **ステージ1：OCR（光学文字認識）**

**モデル：** `gpt-4.1`

| タイプ | トークン数 | レート（100万あたり） | コスト |
| :---- | :---- | :---- | :---- |
| 入力 | 2,000 | $1.00 | $0.002 |
| 出力 | 1,500 | $4.00 | $0.006 |
| **1,000ページの合計（ステージ1）** |  |  | **$8.00** |

#### **ステージ2：推論**

**モデル：** `o4-mini`

| タイプ | トークン数 | レート（100万あたり） | コスト |
| :---- | :---- | :---- | :---- |
| 入力 | 2,000 | $0.55 | $0.0011 |
| 出力 | 3,000 | $2.20 | $0.0066 |
| **1,000ページの合計（ステージ2）** |  |  | **$7.70** |

#### 総計（1,000ページあたり）：**$15.70**

このコストを1ステージの`o3`デプロイメントと比較してください。同等のトークン使用量とバッチ使用を仮定すると、より強力な推論モデルの追加コストは1,000ページあたり$70になります。

### モニタリングとデプロイメント

重要な指標をログに記録してシステムを監視します：

* モデルごとの`llm_model_used`、`llm_input_tokens`、`llm_output_tokens`、`llm_latency_ms`
* モデルごとの`total_query_latency_ms`、`estimated_query_cost`
* `function_calls_per_document`、`num_email_validation_calls`
* `human_review_required`

設定/環境変数を通じてデプロイメントで使用される特定のモデルバージョン識別子（例：`o4-mini-2025-04-16`）を固定して、サイレントモデル更新による予期しない動作を防ぎます。

## 6. 有用なクックブックとリソース

特定のコンポーネントの詳細については、これらの関連リソースを参照してください：

* [構造化出力](https://platform.openai.com/docs/guides/structured-outputs)
* [ビジョンモデル](https://platform.openai.com/docs/guides/images)
* [関数呼び出し](https://platform.openai.com/docs/guides/function-calling)

================================================================================

<h2 id="prototype-to-production">プロトタイプから本番環境へ</h2>

プロトタイプから本番環境への移行には、慎重な計画と実行が必要です。このチェックリストは、私たちの主要なユースケースから得た知見をもとに、デプロイメントが堅牢で効率的であり、ビジネス目標を満たすことを確実にするための重要なステップを強調しています。

## 🗂️ TL;DR マトリックス

| チェックリスト領域 | 主要な焦点/アクション | 重要な理由 |
| :---- | :---- | :---- |
| **成功基準の定義** | • 測定可能なKPIとSLO（精度、コスト、レイテンシ）を定義。• ターゲットがログを通じて測定可能であることを確認。 | 明確なターゲットを提供し、価値を証明する。 |
| **モデル選択理由の文書化** | • トレードオフに基づいて初期モデルを意図的に選択。• モデル選択の「理由」を文書化。 | 選択を正当化し、将来の更新を支援する。 |
| **堅牢な評価とテスト** | • ゴールデンセットを使用した自動テスト（「評価スイート」）を構築。• 事実性、幻覚、ツールエラーに焦点。• ツールの信頼性とエッジケースをテスト。 | 品質を確保し、リリース前の回帰を防ぐ。 |
| **可観測性とコスト** | • モニタリングとデバッグのための重要なログを実装。• コストガードレール（トークン制限、使用モード）を設定。 | チューニングを可能にし、予算内での支出を維持する。 |
| **安全性とコンプライアンス** | • 安全メカニズム（モデレーションAPI、プロンプト）を使用。• ドメイン固有のコンプライアンスルールを実施。• 高リスク出力にHuman-in-the-Loop（HITL）を義務付け。 | 責任ある運用を確保し、要件を満たす。 |
| **モデル更新とバージョン管理** | • バージョン固定戦略を定義 • 新バージョンのA/Bテストを実装 • ロールバック手順を作成 | 改善を可能にしながら安定性を維持する。 |

1. **成功基準を定量的に定義：** 主要な開発前に「動作する」を超えて測定可能なターゲットに移行する。
   
   * **主要業績指標（KPI）とSLOを設定：** ビジネス価値（例：RAG精度 > 95%、OCRコスト < $X/ページ）とパフォーマンス（例：P95レイテンシ < 1秒、エラー率）の具体的なターゲットを定義。
   * **測定可能性を確保：** すべてのKPIとSLOがシステムログから直接測定できることを確認（例：`total_tokens`、`critique_status`の追跡）。

2. **初期モデル選択の理由を文書化：** 将来の参考のためにモデル選択を正当化する。
   
   * **意図的にモデルを選択：** モデル紹介マトリックスとユースケースを使用して、各タスクに適切なモデルを選択（例：速度/コストのための`o4-mini`、精度のための`gpt-4.1`、深度のための`o3`）。
   * **「理由」を記録：** 将来のチームが文脈を理解できるよう、選択の理由（コスト、レイテンシ、機能のトレードオフ）をコードコメントまたは設計文書に簡潔に文書化。

3. **堅牢な評価とテストを実装：** 変更を出荷する前に品質を検証し、回帰を防ぐ。
   
   * **自動評価スイートを構築：** 「ゴールデンセット」（50-100の多様で専門家検証済みの例）を使用した反復可能なテストプロセスを作成。`事実性`、`幻覚率`、`ツールエラー率`、およびタスク固有の指標にテストを集中。
   * **信頼性のあるテスト：** 統合ツールの信頼性（成功率、エラー処理）と負荷およびエッジケース（不正な形式のデータ、敵対的入力）でのシステム動作を厳密にテスト。

4. **可観測性とコスト制御を確立：** パフォーマンスを監視し、予算内で支出を維持する。
   
   * **コストガードレールを設定：** ステージごとの最大トークン制限を定義し、コストとパフォーマンスのバランスを取るための運用モード（「高速」、「標準」、「徹底」）を検討することで、予期しないコスト増加を防ぐ。
   * **重要なログを実装：** デバッグと監視を可能にするため、各処理ステージの重要な運用データを構造化ログを通じて取得。

5. **安全性とコンプライアンスガードレールを実装：** 責任ある運用を確保し、要件を満たす。
   
   * **安全メカニズムを使用：** 特にユーザー入力や機密トピックについて、OpenAIのモデレーションAPI、安全性重視のシステムプロンプト、またはチェック用のセンチネルモデルなどのツールを使用。
   * **コンプライアンスを実施：** 特定の業界とリスクに関連するチェックを組み込む（例：法的制約、実験室安全）。
   * **Human-in-the-Loop（HITL）を要求：** 信頼度の低い出力、高リスクシナリオ、または重要な決定について人間のレビューを義務付け、ワークフローがこれらの項目を明確にフラグ付けすることを確保。

6. **モデル更新とバージョン管理を管理：** 時間の経過とともにモデルの進化に備える。
   
   * **バージョン固定戦略：** 安定性のために特定のモデルバージョンに固定するか、改善のために新しいバージョンを自動的に採用するかを決定。
   * **A/Bテストフレームワーク：** 完全なデプロイメント前に新しいモデルバージョンを主要指標に対して評価するプロセスを確立。
   * **ロールバック計画：** 更新で問題が発生した場合に以前のモデルバージョンに戻すための明確な手順を作成。
   * **バージョンパフォーマンスの監視：** モデルバージョン間で指標を追跡し、パフォーマンストレンドを特定し、将来の選択決定に情報を提供。

================================================================================

## 適応決定ツリー

![モデル選択決定ツリー](../../../images/3D_model_selection_flowchart.png)

## 非技術系ステークホルダーへのモデル選択の説明

ビジネスステークホルダーにモデル選択を説明する際は、以下の重要なポイントに焦点を当ててください：

1. **ビジネス成果との整合性**：モデル選択が特定のビジネス目標（時間節約、コスト削減、精度向上）をどのように直接支援するかを説明。

2. **技術指標をビジネスインパクトに翻訳**：技術的考慮事項をビジネスインパクトに変換：
   - 「このモデルは処理時間を5秒から0.7秒に短縮し、顧客問い合わせを7倍高速に処理できます」
   - 「miniバリアントを使用することで、同じ予算内で5倍多くの文書を処理できます」

3. **トレードオフを強調**：異なるモデルの明確なシナリオを提示：
   - 「オプションA（GPT-4.1）：最高精度だが高コスト - 顧客向け法的分析に最適」
   - 「オプションB（GPT-4.1 mini）：30%のコストで90%の精度 - 内部文書処理に最適」

4. **具体例を使用**：各オプションの価値提案を説明するために、モデル間の出力の実際的な違いを実証。

================================================================================

## 付録

## 主要用語の用語集

| 用語 | 定義 |
|------|------------|
| **コンテキストウィンドウ** | モデルが単一のリクエストで処理できるトークンの最大数 |
| **幻覚** | モデルがもっともらしく見えるが事実的に間違っているか根拠のないコンテンツを生成すること |
| **レイテンシ** | モデルにリクエストを送信してから応答を受信するまでの時間遅延 |
| **LLM** | Large Language Model；膨大なテキストデータで訓練されたAIシステム |
| **プロンプトエンジニアリング** | AIモデルから望ましい出力を引き出すための効果的なプロンプトを設計する実践 |
| **RAG** | Retrieval-Augmented Generation；情報検索とテキスト生成を組み合わせること |
| **SOTA** | State-of-the-Art；特定の時点での分野の最も先進的な段階を表す |
| **トークン** | モデルが処理するテキストの基本単位（英語では約0.75語） |

## 6.1 価格とユーティリティテーブル（2025年4月）

| モデル | コンテキストウィンドウ | 入力価格（100万トークンあたり） | 出力価格（100万トークンあたり） | 最適用途 |
|-------|----------------|-----------------------------|-----------------------------|----------|
| GPT-4.1 | 1M | \$2.00 | \$8.00 | 長文書分析、コードレビュー |
| GPT-4.1 mini | 1M | \$0.40 | \$1.60 | 本番エージェント、バランスの取れたコスト/パフォーマンス |
| GPT-4.1 nano | 1M | \$0.10 | \$0.40 | 高スループット、コスト重視アプリケーション |
| GPT-4o | 128K | \$5.00 | \$15.00 | リアルタイム音声/ビジョンチャット |
| GPT-4o mini | 128K | \$0.15 | \$0.60 | ビジョンタスク、迅速な分析 |
| o3 (low) | 200K | \$10.00* | \$40.00* | バルクトリアージ、カタログ充実化 |
| o3 (med) | 200K | \$10.00* | \$40.00* | ナレッジベースQ&A |
| o3 (high) | 200K | \$10.00* | \$40.00* | 多段階推論、トラブルシューティング |
| o4-mini (low) | 200K | \$1.10* | \$4.40* | ビジョンタスク、迅速な分析 |
| o4-mini (med) | 200K | \$1.10* | \$4.40* | バランスの取れたビジョン + 推論 |
| o4-mini (high) | 200K | \$1.10* | \$4.40* | コスト制御での深い推論 |

\* *注：low/med/high設定は基本価格ではなくトークン使用量に影響します。高い設定では、より深い推論のためにより多くのトークンを使用し、リクエストあたりのコストとレイテンシが増加する可能性があります。*

## 6.2 プロンプトパターンクイックシート（トークン vs レイテンシの差分）

| プロンプトパターン | 説明 | トークンへの影響 | レイテンシへの影響 | 最適なモデル |
|----------------|-------------|--------------|----------------|----------------|
| **自己批評** | 最終化前にモデルに自身の回答を評価させる | +20-30% トークン | +15-25% レイテンシ | GPT-4.1、o3 |
| **思考の連鎖（CoT）** | 「段階的に考える」ことを明示的に指示 | +40-80% トークン | +30-50% レイテンシ | o3、o4-mini (high) |
| **構造化出力** | 一貫したフォーマットのためにJSONスキーマやpydanticモデルを使用 | +5-10% トークン | +5-10% レイテンシ | 全モデル |
| **ゼロトークンメモリ** | 会話内ではなく外部DBにコンテキストを保存 | -70-90% トークン | -5-10% レイテンシ | GPT-4.1ファミリー |
| **スケルトン埋め込み** | モデルが完成するためのテンプレート構造を提供 | -10-20% トークン | -5-15% レイテンシ | o4-mini、GPT-4.1 nano |
| **自己一貫性** | 複数の回答を生成し、最も一貫したものを選択 | +200-300% トークン | +150-250% レイテンシ | o3 (high) |
| **ロールプレイング** | 専門知識のためにモデルに特定のペルソナを割り当て | +5-15% トークン | 中立 | GPT-4o、o4-mini |
| **トーナメントランキング** | 個別にスコア付けするのではなく、オプションをペアワイズで比較 | +50-100% トークン | +30-60% レイテンシ | o3、o4-mini (high) |
| **ツール呼び出し反射** | 不確実性が検出されたときにツールを呼び出すようモデルにプロンプト | +10-30% トークン | +20-40% レイテンシ | o3、GPT-4.1 |

## 6.3 外部クックブックと文書へのリンク

### OpenAI公式リソース
- [OpenAI Cookbookメインリポジトリ](https://cookbook.openai.com/)
- [関数呼び出しガイド](https://platform.openai.com/docs/guides/function-calling)
- [ビジョンモデルガイド](https://platform.openai.com/docs/guides/vision)
- [エージェント文書](https://platform.openai.com/docs/guides/agents)
- [構造化出力ガイド](https://platform.openai.com/docs/guides/structured-outputs)

### RAGと検索
- [PDFでのRAG](https://cookbook.openai.com/examples/file_search_responses)

### 専門的なユースケース
- [Agents SDKを使用した音声アシスタント](https://cookbook.openai.com/examples/agents_sdk/app_assistant_voice_agents)
- [マルチツールオーケストレーション](https://cookbook.openai.com/examples/responses_api/responses_api_tool_orchestration)
- [データ抽出と変換](https://cookbook.openai.com/examples/data_extraction_transformation)

### プロンプティングとモデル選択
- [GPT-4.1プロンプティングガイド](https://cookbook.openai.com/examples/gpt4-1_prompting_guide)
- [プロンプトエンジニアリングベストプラクティス](https://platform.openai.com/docs/guides/prompt-engineering)

### 評価とデプロイメント
- [OpenAI Evalsの開始方法](https://cookbook.openai.com/examples/evaluation/getting_started_with_openai_evals)
- [Usage APIとCost APIを使用してOpenAIの使用量を監視する方法](https://cookbook.openai.com/examples/completions_usage_api)

================================================================================

## 貢献者

このクックブックは、OpenAIと[Tribe AI](https://www.tribe.ai/)の共同協力の成果です。
- [Kashyap Coimbatore Murali](https://www.linkedin.com/in/kashyap-murali/)
- [Nate Harada](https://www.linkedin.com/in/nate-harada/) 
- [Sai Prashanth Soundararaj](https://www.linkedin.com/in/saiprashanths/)
- [Shikhar Kwatra](https://www.linkedin.com/in/shikharkwatra/)